In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

#data preprocessing
data_dir = "/kaggle/input/new-cat-dataset/extracted_frames_for_my_video"  
image_size = (128, 128)  # Resizing of frames

def load_images(data_dir, image_size):
    images = []
    sorted_files = sorted(os.listdir(data_dir))
    for file in sorted_files:
        img_path = os.path.join(data_dir, file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, image_size)
            images.append(img)
    return np.array(images)


images = load_images(data_dir, image_size)
images = images / 255.0  # Normalizing the pixel values to [0, 1]

# Create sequences for prediction
X = images[:-1]
y = images[1:]

# print(X)
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=42)

# To check if images are loaded
print(f"Total frames loaded: {images.shape}")

# Ensure there's data to split
if len(X) == 0 or len(y) == 0:
    raise ValueError("No data available for splitting. Check your dataset directory and image loading logic.")

# Train-test split with adjusted test size
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
print(f"Train samples: {len(X_train)}, Test samples: {len(X_test)}")

In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128 * 128 * 3, activation='sigmoid'),
    layers.Reshape((128, 128, 3))
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_21 (Conv2D)                   │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_17 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_22 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_18 (MaxPooling2D)      │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 256)                 │      14,745,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 49152)               │      12,632,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_8 (Reshape)                  │ (None, 128, 128, 3)         │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 27,397,312 (104.51 MB)

 Trainable params: 27,397,312 (104.51 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Training the model with these parameters
history = model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_test, y_test),
    batch_size=8
)

Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 216ms/step - loss: 0.0366 - mae: 0.1558 - val_loss: 0.0285 - val_mae: 0.1349
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0263 - mae: 0.1295 - val_loss: 0.0269 - val_mae: 0.1307
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0251 - mae: 0.1261 - val_loss: 0.0265 - val_mae: 0.1293
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0247 - mae: 0.1248 - val_loss: 0.0256 - val_mae: 0.1274
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0241 - mae: 0.1229 - val_loss: 0.0218 - val_mae: 0.1168
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0191 - mae: 0.1083 - val_loss: 0.0173 - val_mae: 0.1009
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0163 - mae: 0.0971 - val_loss: 0.0173 - val_mae: 0.0984
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0154 - mae: 0.0945 - val_loss: 0.0164 - val_mae: 0.0971
Epoch 9/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.01

In [ ]:
#testing the model against the test dataset
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")


import matplotlib.pyplot as plt

def visualize_predictions(model, X_test, y_test, num_samples=5):
    predictions = model.predict(X_test[:num_samples])
    for i in range(num_samples):
        plt.figure(figsize=(10, 4))

        # Original next frame
        plt.subplot(1, 3, 1)
        plt.imshow(y_test[i])
        plt.title("Actual Frame")

        # Input frame
        plt.subplot(1, 3, 2)
        plt.imshow(X_test[i])
        plt.title("Input Frame")

        # Predicted next frame
        plt.subplot(1, 3, 3)
        plt.imshow(predictions[i])
        plt.title("Predicted Frame")

        plt.show()

visualize_predictions(model, X_test, y_test)